In [1]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import random
from multiprocessing import Process
import math
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
import undetected_chromedriver as uc

In [2]:
domain = "https://www.vocabulary.com/"

In [3]:
def get_url(path):
    return domain + path

In [39]:
global_driver = uc.Chrome()
# driver.implicitly_wait(5)

In [86]:
# service = Service(executable_path="../chromedriver")
# driver = webdriver.Chrome(service=service)

In [4]:
USER_AGENTS = [
     "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    
    # Chrome on macOS
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
]


In [5]:
def init_driver():
    chrome_options = uc.ChromeOptions()
    # chrome_options.add_argument('--headless')  # Run in headless mode
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    # chrome_options.add_argument('--window-size=1920,1080')  # Set window size
    chrome_options.add_argument('--disable-gpu')  # Disable GPU hardware acceleration
    # chrome_options.add_argument(f"user-agent={random.choice(USER_AGENTS)}")  # Randomize User-Agent
    
    driver = uc.Chrome(options=chrome_options, user_multi_procs=True)
    driver.set_page_load_timeout(10)

    return driver
    
    

In [45]:
init_driver()

<undetected_chromedriver.Chrome (session="b9575dfd6c24495e4a978c5b70096c11")>

In [6]:
def get_request(url, driver, save_path=None, wait=None):
    # Set up ChromeDriver service
    # service = Service(executable_path="../chromedriver")
    
    # Initialize the driver
    # driver = webdriver.Chrome(service=service, options=chrome_options)
    
    try:
        driver.get(url)
    except Exception as e:
        print(f"Error loading page: {e}")
        
        # Retry loading the page
        try:
            print('Retrying after 5 seconds ...')
            sleep(5)
            driver.get(url)
        except Exception as e:
            print(f"Retry failed: {e}")
            raise e
            
    # Wait for Cloudflare check to complete
    if "Cloudflare" in driver.page_source:
        print("Cloudflare verification...")
        sleep(10)
    
    if wait:
        sleep(wait)
        
    WebDriverWait(driver, 20).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
    
    # Parse HTML
    soup = BeautifulSoup(driver.page_source, "html.parser")
    if save_path:
        with open(save_path, "w", encoding='utf-8') as f:
            f.write(soup.prettify())
    
    return soup
    
    # finally:
    #     driver.quit()

# 1. Crawl collection links

## 1.1 Crawl topic "lists-by-grade"

In [87]:
soup = get_request(domain + "/lists/lists-by-grade", "lists-by-grade.html")

### 1.1.1 Get all grades and links to each grade page

In [40]:
# Get all the a tags with class ".lk-nav-by-grade"
grade_a = soup.find_all("a", class_="lk-nav-by-grade")

grade_links = [ {"Grade":grade_a.text, "Link": grade_a.get("href")} for grade_a in grade_a ]

grade_links_df = pd.DataFrame(grade_links)
grade_links_df.head()

,Grade,Link
0,Grade 5,/lists/lists-by-grade/grade-5
1,Grade 6,/lists/lists-by-grade/grade-6
2,Grade 7,/lists/lists-by-grade/grade-7
3,Grade 8,/lists/lists-by-grade/grade-8
4,Grade 9,/lists/lists-by-grade/grade-9


In [41]:
grade_links_df.to_csv("lists-by-grade/grade_links.csv", index=False)

### 1.1.2. Write solution crawling a certain grade

In [9]:
grade_links = pd.read_csv("lists-by-grade/grade_links.csv")
grade_links.head(2)

,Grade,Link
0,Grade 5,/lists/lists-by-grade/grade-5
1,Grade 6,/lists/lists-by-grade/grade-6


In [73]:
def fetch_vocaset_by_category_url(category_url):
    print(f"Fetching collections from base url: {category_url}")
    
    soup = get_request(category_url, wait=5)
    
    # Get super collections
    collection_selectors = [
        '.item-vocab-list:not(.is-image)', # Grade
        '.unit-list .type_work', # Literature, Non-Fiction
        '.unit-list .type_collection.style-feature', # Literature, Non-Fiction
        '.unit-list .type_collection.style-items-hidden' # Test-Prep
    ]
    
    vocasets = soup.select(', '.join(collection_selectors))  
    vocaset_data = []
    for vocaset in vocasets:
        name = vocaset.find("a").text.strip()
        link = vocaset.find("a")["href"]
        
        if not link:
            continue
        
        voca_set_soup = get_request(get_url(link))
        
        child_voca_sets = voca_set_soup.select(".unit.type_work.with-icon")
        child_voca_sets_2 = voca_set_soup.select(".unit.type_collection.with-icon")
        
        if child_voca_sets or child_voca_sets_2:
            for child_voca_set in child_voca_sets:
                child_name = child_voca_set.select_one(".details a .title").text.strip()
                child_link  = child_voca_set.select_one(".details a")["href"]
                
                if child_link:
                    vocaset_data.append({
                        "Name": child_name,
                        "Link": child_link
                    })
                    print('Sub vocaset type 1: ', child_name, child_link)
            
            for child_voca_set in child_voca_sets_2:
                child_name = child_voca_set.select_one(".details a .title").text.strip()
                child_link  = child_voca_set.select_one(".details a")["href"]
                
                if child_link:
                    vocaset_data.append({
                        "Name": child_name,
                        "Link": child_link
                    })
                    print('Sub vocaset type 2: ', child_name, child_link)
        elif voca_set_soup.select_one(".flashcard-panel .list-selector"):
            vocaset_data.append({
                "Name": name,
                "Link": link
            })
            print('Vocaset: ', name, link)
        
    return vocaset_data

In [44]:
grade = grade_links.iloc[0]
grade

Grade                          Grade 5
Link     /lists/lists-by-grade/grade-5
Name: 0, dtype: object

In [11]:
result = fetch_vocaset_by_category_path(grade_links.iloc[0]["Link"])
len(result)

Vocaset:  150 Words Every 5th Grader Should Know /lists/2v2w6pjn/150-words-every-5th-grader-should-know
Vocaset:  Commonly Confused Words, Grade 5 /lists/5khonlqz/commonly-confused-words-grade-5
Vocaset:  Tricky Homonyms for Students /lists/1108220
Vocaset:  Common Core Vocabulary /lists/6z3umhtf/grade-5
Vocaset:  Foreign Words and Phrases /lists/7mdntuqn/foreign-words-and-phrases-commonly-used-by-english-speakers
Sub vocaset type 1:  A Wrinkle in Time /lists/acfgpvmv/wrinkle-in-time
Sub vocaset type 1:  The One and Only Ivan /lists/wwhbu4wt/one-and-only-ivan
Sub vocaset type 1:  I Can Make This Promise /lists/vilvbakt/i-can-make-this-promise
Sub vocaset type 1:  The Borrowers /lists/4rp5gas2/borrowers
Sub vocaset type 1:  Treasure Island /lists/dtafbzbr/treasure-island
Sub vocaset type 1:  Maniac Magee /lists/p3xg2wzr/maniac-magee
Sub vocaset type 1:  Harry Potter and the Sorcerer's Stone /lists/intdwbk3/harry-potter-and-the-sorcerers-stone
Sub vocaset type 1:  The Bad Beginning /list

377

### 1.1.3. Crawl and export voca sets of all grade

In [29]:
grade_links.head(2)

,Grade,Link
0,Grade 5,/lists/lists-by-grade/grade-5
1,Grade 6,/lists/lists-by-grade/grade-6


In [30]:
for grade_index in range(len(grade_links)):
    grade = grade_links.iloc[grade_index]
    print(grade)
    grade_name = grade["Grade"]
    grade_url = get_url(grade["Link"])
    
    print('------------------\n')
    print(f"Fetching vocasets for grade: {grade_name}")
    
    voca_sets = fetch_vocaset_by_category_path(grade["Link"])
    voca_sets_df = pd.DataFrame(voca_sets)
    voca_sets_df.to_csv(f"lists-by-grade/{grade_name}.csv", index=False)
    
    print('-------------------\n')
    print(f'Done fetching {grade_name}, total voca sets : {len(voca_sets)}')
    sleep(4)

Grade                          Grade 5
Link     /lists/lists-by-grade/grade-5
Name: 0, dtype: object
------------------

Fetching vocasets for grade: Grade 5
Vocaset:  150 Words Every 5th Grader Should Know /lists/2v2w6pjn/150-words-every-5th-grader-should-know
Vocaset:  Commonly Confused Words, Grade 5 /lists/5khonlqz/commonly-confused-words-grade-5
Vocaset:  Common Core Vocabulary /lists/6z3umhtf/grade-5
Vocaset:  Foreign Words and Phrases /lists/7mdntuqn/foreign-words-and-phrases-commonly-used-by-english-speakers
Sub vocaset type 1:  A Wrinkle in Time /lists/acfgpvmv/wrinkle-in-time
Sub vocaset type 1:  The One and Only Ivan /lists/wwhbu4wt/one-and-only-ivan
Sub vocaset type 1:  I Can Make This Promise /lists/vilvbakt/i-can-make-this-promise
Sub vocaset type 1:  The Borrowers /lists/4rp5gas2/borrowers
Sub vocaset type 1:  Treasure Island /lists/dtafbzbr/treasure-island
Sub vocaset type 1:  Maniac Magee /lists/p3xg2wzr/maniac-magee
Sub vocaset type 1:  Harry Potter and the Sorcerer's

### 1.1.4. Look some statistic value

In [31]:
grade_dfs = []

for grade_index in range(len(grade_links)):
    grade = grade_links.iloc[grade_index]
    print(grade)
    grade_name = grade["Grade"]
    
    grade_df = pd.read_csv(f"lists-by-grade/{grade_name}.csv")
    
    # Add category column
    grade_df["Category"] = grade_name
    grade_dfs.append(grade_df)
    
# Concatenate all DataFrames
all_grade_df = pd.concat(grade_dfs, ignore_index=True)

all_grade_df.to_csv("lists-by-grade/all_grade_vocasets.csv", index=False)

all_grade_df.head(2)

Grade                          Grade 5
Link     /lists/lists-by-grade/grade-5
Name: 0, dtype: object
Grade                          Grade 6
Link     /lists/lists-by-grade/grade-6
Name: 1, dtype: object
Grade                          Grade 7
Link     /lists/lists-by-grade/grade-7
Name: 2, dtype: object
Grade                          Grade 8
Link     /lists/lists-by-grade/grade-8
Name: 3, dtype: object
Grade                          Grade 9
Link     /lists/lists-by-grade/grade-9
Name: 4, dtype: object
Grade                          Grade 10
Link     /lists/lists-by-grade/grade-10
Name: 5, dtype: object
Grade                          Grade 11
Link     /lists/lists-by-grade/grade-11
Name: 6, dtype: object
Grade                          Grade 12
Link     /lists/lists-by-grade/grade-12
Name: 7, dtype: object


,Name,Link,Category
0,150 Words Every 5th Grader Should Know,/lists/2v2w6pjn/150-words-every-5th-grader-sho...,Grade 5
1,"Commonly Confused Words, Grade 5",/lists/5khonlqz/commonly-confused-words-grade-5,Grade 5


In [30]:
def join_category(df):
    deduped_collections = df.drop_duplicates(["Link", "Category"])
    deduped_collections = deduped_collections.groupby(["Link"])["Category"].apply(lambda x: ','.join(x)).reset_index()
    deduped_collections["Number Of Categories"] = deduped_collections["Category"].apply(lambda x: len(x.split(',')))
    return deduped_collections

In [31]:
all_grade_voca_sets = pd.read_csv("lists-by-grade/all_grade_vocasets.csv")
len(all_grade_voca_sets)

5215

In [38]:
all_grade_voca_sets.head(2)

,Name,Link,Category
0,150 Words Every 5th Grader Should Know,/lists/2v2w6pjn/150-words-every-5th-grader-sho...,Grade 5
1,"Commonly Confused Words, Grade 5",/lists/5khonlqz/commonly-confused-words-grade-5,Grade 5


In [32]:
deduped_collections = join_category(all_grade_voca_sets)
deduped_collections.head(2)

,Link,Category,Number Of Categories
0,/lists/22f5x52u/bunnicula,"Grade 6,Grade 7,Grade 8",3
1,/lists/25x6nyvl/absolutely-normal-chaos,Grade 8,1


In [33]:
deduped_collections["Number Of Categories"].value_counts()

Number Of Categories
1    277
7    237
4    207
3    108
2     79
8     48
6     37
5     15
Name: count, dtype: int64

In [35]:
deduped_collections["Link"].nunique()

1008

In [36]:
len(deduped_collections["Link"])

1008

In [37]:
deduped_collections.to_csv("lists-by-grade/unique_collection.csv", index=False)

### 1.1.6. Crawl collection (vocaset) details

In [14]:
def get_collection_details(collection_path):
    collection_soup = get_request(get_url(collection_path))
    
    title = collection_soup.select_one(".summary-panel .title-actions-stats .title .title").contents[0].strip()
    description = collection_soup.select_one(".summary-panel .title-actions-stats blockquote").text.strip()
    thumbnail_link = collection_soup.select_one(".summary-panel .summary .header-img img")["src"]
    
    book_purchase_link_a_tag = collection_soup.select_one(".summary-panel .title-actions-stats a.buy-book")
    book_purchase_link = book_purchase_link_a_tag["href"] if book_purchase_link_a_tag else None
    
    author_a_tag = collection_soup.select_one(".summary-panel .title-actions-stats .authors a")
    author = author_a_tag.text.strip() if author_a_tag else None
    
    return {
        "Name": title,
        "Description": description,
        "Thumbnail Link": thumbnail_link,
        "Book Purchase Link": book_purchase_link,
        "Author": author  
    }

In [20]:
all_collection_df = pd.read_csv("lists-by-grade/unique_collection.csv")
all_collection_df.head(2)

,Name,Link,Category
0,150 Words Every 5th Grader Should Know,/lists/2v2w6pjn/150-words-every-5th-grader-sho...,Grade 5
1,"Commonly Confused Words, Grade 5",/lists/5khonlqz/commonly-confused-words-grade-5,Grade 5


In [21]:
get_collection_details(all_collection_df.iloc[0]["Link"])

{'Name': '150 Words Every 5th Grader Should Know',
 'Description': "By analyzing student data and commonly taught texts, Vocabulary.com has compiled this collection of essential vocabulary for students in 5th grade. Master high-frequency, high-utility words that you'll encounter across the curriculum.",
 'Thumbnail Link': 'https://cdn.vocabulary.com/units/2v2w6pjn/feature.png?width=500&v=17d33615117',
 'Book Purchase Link': None,
 'Author': None}

In [23]:
collection_details = []

for index in range(2):
    collection = all_collection_df.iloc[index]
    
    info = get_collection_details(collection["Link"])
    info["Category"] = collection["Category"]
    info["Link"] = collection["Link"]
    
    collection_details.append(info)
    print(f"Fetched details for {collection['Name']}")

Fetched details for 150 Words Every 5th Grader Should Know
Fetched details for Commonly Confused Words, Grade 5


In [24]:
collection_details

[{'Name': '150 Words Every 5th Grader Should Know',
  'Description': "By analyzing student data and commonly taught texts, Vocabulary.com has compiled this collection of essential vocabulary for students in 5th grade. Master high-frequency, high-utility words that you'll encounter across the curriculum.",
  'Thumbnail Link': 'https://cdn.vocabulary.com/units/2v2w6pjn/feature.png?width=500&v=17d33615117',
  'Book Purchase Link': None,
  'Author': None,
  'Category': 'Grade 5',
  'Link': '/lists/2v2w6pjn/150-words-every-5th-grader-should-know'},
 {'Name': 'Commonly Confused Words, Grade 5',
  'Description': 'Commonly confused words are pairs or groups of words that share similar spellings or pronunciations but have different meanings or usages. This collection features commonly confused words that fifth-grade students frequently encounter while reading or taking standardized tests.',
  'Thumbnail Link': 'https://cdn.vocabulary.com/units/5khonlqz/feature.png?width=500&v=18e1a777fc2',
  'B

In [25]:
get_collection_details("/lists/nvicmlqn/call-us-what-we-carry")

{'Name': 'Call Us What We Carry',
 'Description': "This collection of poetry reflects on more than a century's worth of American history, exploring a variety of topics including racism, the 1918 influenza pandemic, climate change, and the AIDS epidemic.",
 'Thumbnail Link': 'https://cdn.vocabulary.com/units/nvicmlqn/cover.jpeg?width=220&v=1800b30b02b',
 'Book Purchase Link': 'https://www.amazon.com/dp/0593465067?tag=vocabulary01-20',
 'Author': 'Amanda Gorman'}

## 1.2. Crawl collection of "Literature"

In [27]:
literature_path = '/lists/literature/collections'

In [35]:
voca_sets = fetch_vocaset_by_category_url(get_url(literature_path))

len(voca_sets)

Fetching collections from base url: https://www.vocabulary.com//lists/literature/collections
Vocaset:   /lists/g7qeq2te/heroes-of-the-water-monster
Vocaset:   /lists/vxlwpv4i/strange-birds
Sub vocaset type 1:  The Wishing Spell /lists/g5wlzkw7/wishing-spell
Sub vocaset type 1:  Where the Mountain Meets the Moon /lists/dw3zvtpy/where-the-mountain-meets-the-moon
Sub vocaset type 1:  There’s Something about Sweetie /lists/vmpuxvgn/theres-something-about-sweetie
Sub vocaset type 1:  The Invention of Hugo Cabret /lists/g4ihj4xd/invention-of-hugo-cabret
Sub vocaset type 1:  What If It's Us /lists/77luls7n/what-if-its-us
Sub vocaset type 1:  The Cardboard Kingdom /lists/2dwh5nrs/cardboard-kingdom
Sub vocaset type 1:  The Vanderbeekers of 141st Street /lists/37woga72/vanderbeekers-of-141st-street
Sub vocaset type 1:  A Tangle of Knots /lists/sa45o3qs/tangle-of-knots
Sub vocaset type 1:  Better Nate Than Ever /lists/hcxq22fu/better-nate-than-ever
Sub vocaset type 1:  Heidi /lists/4vb7hfxz/heidi

1993

In [37]:
voca_sets_df = pd.DataFrame(voca_sets)
voca_sets_df.to_csv(f"literature/Literature.csv", index=False)

In [38]:
unique_collection_df = voca_sets_df.drop_duplicates(subset='Name')
len(unique_collection_df)

1068

In [43]:
unique_collection_df.loc[:, 'Category'] = "Literature"
unique_collection_df.to_csv("literature/unique_collection.csv", index=False)
unique_collection_df.head(2)

,Name,Link,Category
0,,/lists/g7qeq2te/heroes-of-the-water-monster,Literature
2,The Wishing Spell,/lists/g5wlzkw7/wishing-spell,Literature


## 1.3. Crawl collection of "Non-fiction"

In [56]:
non_fiction_path = "lists/non-fiction/collections"

In [64]:
collection_data = fetch_vocaset_by_category_url(get_url(non_fiction_path))
len(collection_data)

Fetching collections from base url: https://www.vocabulary.com/lists/non-fiction/collections
Vocaset:   /lists/tln3hix5/fallout
Vocaset:   /lists/tl4r6xaq/camp-panda
Vocaset:   /lists/nnbem5ba/reaching-for-the-moon
Sub vocaset type 1:  A Face for Picasso /lists/mgrppyte/face-for-picasso
Sub vocaset type 1:  I Am Malala /lists/kkjix4tr/i-am-malala
Sub vocaset type 1:  Almost American Girl /lists/mgv3reo3/almost-american-girl
Sub vocaset type 1:  Educated /lists/4l4dqxks/educated
Sub vocaset type 1:  I Know Why the Caged Bird Sings /lists/mp532xvv/i-know-why-the-caged-bird-sings
Sub vocaset type 1:  Apple /lists/2ljn37ry/apple
Sub vocaset type 1:  Brown Girl Dreaming /lists/esrg7ttd/brown-girl-dreaming
Sub vocaset type 1:  The Glass Castle /lists/ewxdgzaz/glass-castle
Sub vocaset type 1:  Born a Crime /lists/fgyutrhj/born-a-crime
Sub vocaset type 1:  Hey, Kiddo /lists/ljyioimr/hey-kiddo
Sub vocaset type 1:  Hunger of Memory /lists/yjhwbxuq/hunger-of-memory
Sub vocaset type 1:  Kaffir Boy

660

In [66]:
collection_df = pd.DataFrame(collection_data)
collection_df.head(2)

,Name,Link
0,,/lists/tln3hix5/fallout
1,,/lists/tl4r6xaq/camp-panda


In [68]:
collection_df.loc[:, "Category"] = "Non-Fiction"
collection_df.head()


,Name,Link,Category
0,,/lists/tln3hix5/fallout,Non-Fiction
1,,/lists/tl4r6xaq/camp-panda,Non-Fiction
2,,/lists/nnbem5ba/reaching-for-the-moon,Non-Fiction
3,A Face for Picasso,/lists/mgrppyte/face-for-picasso,Non-Fiction
4,I Am Malala,/lists/kkjix4tr/i-am-malala,Non-Fiction


In [69]:
len(collection_df)

660

In [70]:
unique_collection_df = collection_df.drop_duplicates(subset='Link')
len(unique_collection_df)

375

In [71]:
unique_collection_df.to_csv("non-fiction/unique_collection.csv", index=False)

## 1.4. Crawl collection of "Test-Prep"

In [74]:
test_prep_path = "lists/test-prep/collections"

In [ ]:
collection_data = fetch_vocaset_by_category_url(get_url(test_prep_path))
len(collection_data)

In [77]:
collection_df = pd.DataFrame(collection_data)
collection_df.head(2)


,Name,Link
0,,/lists/e7i2wuj4/american-history-government-an...
1,,/lists/ln7kok2u/essential-science-vocabulary


In [78]:
len(collection_df)

54

In [79]:
collection_df.loc[:, "Category"] = "Test-Prep"
collection_df.head()

,Name,Link,Category
0,,/lists/e7i2wuj4/american-history-government-an...,Test-Prep
1,,/lists/ln7kok2u/essential-science-vocabulary,Test-Prep
2,,/lists/qt6iiywn/essential-math-vocabulary,Test-Prep
3,The GRE Verbal Reasoning Test: Warm-up Words,/lists/upui5gu6/gre-verbal-reasoning-test-warm...,Test-Prep
4,The GRE Verbal Reasoning Test: Intermediate Words,/lists/cbb7njl3/gre-verbal-reasoning-test-inte...,Test-Prep


In [80]:
unique_collection_df = collection_df.drop_duplicates(subset='Link')
len(unique_collection_df)

38

In [81]:
unique_collection_df.to_csv("test-prep/unique_collection.csv", index=False)

## 1.5. Crawl collection of "Events"

In [82]:
event_path = "lists/current-events/collections"

In [ ]:
collection_data = fetch_vocaset_by_category_url(get_url(event_path))
len(collection_data)

In [85]:
collection_df = pd.DataFrame(collection_data)
collection_df.loc[:, "Category"] = "Events"
unique_collection_df = collection_df.drop_duplicates(subset='Link')

In [86]:
len(unique_collection_df)

71

In [87]:
unique_collection_df.head(2)

,Name,Link,Category
0,,/lists/5xz3zovp/ripped-from-the-headlines-apri...,Events
1,,/lists/h7l2wmzf/ripped-from-the-headlines-marc...,Events


In [89]:
unique_collection_df.to_csv("events/unique_collection.csv", index=False)

## 1.6. Crawl collection of "Roots & Affixes"

In [91]:
root_and_affix_path = "lists/roots-and-affixes/collections"

In [92]:
collection_data = fetch_vocaset_by_category_url(get_url(root_and_affix_path))
len(collection_data)

Fetching collections from base url: https://www.vocabulary.com/lists/roots-and-affixes/collections
Vocaset:   /lists/z7oijolz/power-suffixes
Vocaset:   /lists/zhvfrgrz/power-prefixes
Vocaset:   /lists/m5nm36xq/english-words-and-terms-derived-from-french
Vocaset:   /lists/bb4udog5/wide-world-of-word-origins
Sub vocaset type 2:  The Suffix -id /lists/73tjrqek/suffix-id
Sub vocaset type 2:  The Suffix -or /lists/q5a3yyd7/its-the-end-of-the-word-as-we-know-it-the-suffix-or
Sub vocaset type 2:  The Suffix -ism /lists/2wmnhqzo/suffix-ism
Sub vocaset type 2:  The Suffix -ish /lists/7qpbfd3x/suffix-ish
Sub vocaset type 2:  The Suffix -ic /lists/pvwygdai/suffix-ic
Vocaset:   /lists/7mdntuqn/foreign-words-and-phrases-commonly-used-by-english-speakers
Vocaset:   /lists/jcmqdia6/essential-word-roots
Vocaset:   /lists/kvnetikm/common-senses
Vocaset:   /lists/ilzl5hfg/we-the-people
Vocaset:   /lists/vboyi3bd/elements-of-the-universe
Vocaset:   /lists/ngnbejaa/body-language
Vocaset:   /lists/pvwygdai

20

In [93]:
collection_df = pd.DataFrame(collection_data)
collection_df.loc[:, "Category"] = "Roots & Affixes"
unique_collection_df = collection_df.drop_duplicates(subset='Link')

In [94]:
len(unique_collection_df)

15

In [95]:
unique_collection_df.to_csv("roots-affixes/unique_collection.csv", index=False)

In [96]:
def get_df_of_collection_by_category(category_path, category_name, output_folder):
    collection_data = fetch_vocaset_by_category_url(get_url(category_path))
    
    collection_df = pd.DataFrame(collection_data)
    collection_df.loc[:, "Category"] = category_name
    unique_collection_df = collection_df.drop_duplicates(subset='Link')
    unique_collection_df.to_csv(f"{output_folder}/unique_collection.csv", index=False)
    
    print(f"Fetched {len(unique_collection_df)} collections for {category_name}")
    
    return unique_collection_df

## 1.7 Crawl collection of "Just for Fun"

In [99]:
j4f_path = "lists/just-for-fun/collections"

In [100]:
j4f_df = get_df_of_collection_by_category(j4f_path, "Just for Fun", "just-for-fun")

Fetching collections from base url: https://www.vocabulary.com/lists/just-for-fun/collections


Vocaset:   /lists/q2l5oe6x/words-to-help-you-win-at-scrabble
Vocaset:   /lists/qnrp4ttb/spring-has-sprung
Vocaset:   /lists/6yr66jyo/tolkien-reading-day
Vocaset:   /lists/b65rhjfp/beast-mode-an-ark-of-animal-words
Vocaset:   /lists/iofdi62y/wide-words-of-sports
Vocaset:   /lists/h3ggo73i/mardi-gras
Vocaset:   /lists/krafc3ft/in-like-a-lion-out-like-a-lamb
Vocaset:   /lists/p6p5xvfn/academy-awards
Vocaset:   /lists/n3wg3knp/said-is-dead
Vocaset:   /lists/n6drqxxv/readysethike
Vocaset:   /lists/lhjcn7kj/love-collection
Vocaset:   /lists/qey3cuz5/short-and-sweet
Vocaset:   /lists/ingmji4i/wintry-mix
Vocaset:   /lists/dpmrw5qm/new-words-resolution
Vocaset:   /lists/4a5kowcd/lets-go-caroling
Vocaset:   /lists/dgioeg5o/words-every-crossword-puzzler-should-know
Sub vocaset type 1:  A Christmas Carol /lists/m3v55ze6/christmas-carol
Sub vocaset type 2:  Let's Go Caroling /lists/4a5kowcd/lets-go-caroling
Vocaset:   /lists/qvh7gvbh/feast-on-this-cornucopia-of-tasty-terms
Vocaset:   /lists/ov4p2j6

In [102]:
j4f_df.head()

,Name,Link,Category
0,,/lists/q2l5oe6x/words-to-help-you-win-at-scrabble,Just for Fun
1,,/lists/qnrp4ttb/spring-has-sprung,Just for Fun
2,,/lists/6yr66jyo/tolkien-reading-day,Just for Fun
3,,/lists/b65rhjfp/beast-mode-an-ark-of-animal-words,Just for Fun
4,,/lists/iofdi62y/wide-words-of-sports,Just for Fun


In [103]:
len(j4f_df)

155

## 1.8. Merge all df

In [105]:
folder_list = [
    "lists-by-grade",
    "literature",
    "non-fiction",
    "test-prep",
    "events",
    "roots-affixes",
    "just-for-fun"
]

In [106]:
df_list = []
for folder in folder_list:
    df = pd.read_csv(f"{folder}/unique_collection.csv")
    df_list.append(df)
    print(f"Fetched {len(df)} collections for {folder}")

Fetched 1008 collections for lists-by-grade
Fetched 1068 collections for literature
Fetched 375 collections for non-fiction
Fetched 38 collections for test-prep
Fetched 71 collections for events
Fetched 15 collections for roots-affixes
Fetched 155 collections for just-for-fun


In [107]:
# Concatenate all DataFrames
all_collections_df = pd.concat(df_list, ignore_index=True)
all_collections_df.head(2)

,Link,Category,Number Of Categories,Name
0,/lists/22f5x52u/bunnicula,"Grade 6,Grade 7,Grade 8",3.0,NaN
1,/lists/25x6nyvl/absolutely-normal-chaos,Grade 8,1.0,NaN


In [109]:
len(all_collections_df)

2730

In [111]:
all_collections_df["Category"].value_counts()

Category
Literature                                                            1068
Non-Fiction                                                            375
Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12             223
Just for Fun                                                           155
Grade 9,Grade 10,Grade 11,Grade 12                                     143
Grade 5                                                                115
Grade 6,Grade 7,Grade 8                                                 95
Events                                                                  71
Grade 5,Grade 6,Grade 7,Grade 8                                         60
Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12      48
Grade 12                                                                38
Test-Prep                                                               38
Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12                      34
Grade 11,Grade 1

In [112]:
joined_category_df = join_category(all_collections_df)
len(joined_category_df)

1740

In [113]:
joined_category_df["Category"].value_counts()

Category
Literature                                                               422
Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,Literature    114
Grade 9,Grade 10,Grade 11,Grade 12,Literature                             77
Non-Fiction                                                               74
Just for Fun                                                              73
                                                                        ... 
Grade 6,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12                         1
Grade 5,Grade 9,Grade 10,Grade 11,Grade 12,Non-Fiction                     1
Grade 6,Grade 7,Grade 8,Test-Prep                                          1
Grade 5,Grade 7,Literature,Non-Fiction                                     1
Grade 9,Grade 10,Grade 12                                                  1
Name: count, Length: 156, dtype: int64

In [117]:
joined_category_df["Link"].isna().sum()

np.int64(0)

In [118]:
joined_category_df.to_csv("collection_links.csv", index=False)
joined_category_df.head(2)

,Link,Category,Number Of Categories
0,/lists/22f5x52u/bunnicula,"Grade 6,Grade 7,Grade 8,Literature,Just for Fun",5
1,/lists/23diw7dx/ripped-from-the-headlines-febr...,Events,1


# 2. Crawl collection details

In [7]:
def fetch_collection_details(collection_path, driver=None):
    collection_soup = get_request(get_url(collection_path), driver=driver)
    
    summary_tag = collection_soup.select_one(".summary-panel .title-actions-stats");  

    title = summary_tag.select_one(".title .title").contents[0].strip()
    
    subtitle_tag = summary_tag.select_one(".subtitle")
    subtitle = subtitle_tag.text.strip() if subtitle_tag else None
    
    description = summary_tag.select_one("blockquote").text.strip()
    
    thumbnail_link = collection_soup.select_one(".summary-panel .summary .header-img img")["src"]
    
    book_purchase_link_a_tag = summary_tag.select_one("a.buy-book")
    book_purchase_link = book_purchase_link_a_tag["href"] if book_purchase_link_a_tag else None
    
    author_a_tag = summary_tag.select_one(".authors a")
    author = author_a_tag.text.strip() if author_a_tag else None
    
    return {
        "Title": title,
        "Subtitle": subtitle,
        "Description": description,
        "Thumbnail Link": thumbnail_link,
        "Book Purchase Link": book_purchase_link,
        "Author": author  
    }

In [26]:
collection_links = pd.read_csv("collection_links.csv")
print("Number of collection links:", len(collection_links))
collection_links.head(2)

Number of collection links: 1740


,Link,Category,Number Of Categories
0,/lists/22f5x52u/bunnicula,"Grade 6,Grade 7,Grade 8,Literature,Just for Fun",5
1,/lists/23diw7dx/ripped-from-the-headlines-febr...,Events,1


In [9]:
def get_collection_details(collection_link_df):
    driver = init_driver()
    collection_details = []
    for index in range(len(collection_link_df)):
        collection = collection_link_df.iloc[index]
        
        try:
            info = fetch_collection_details(collection["Link"], driver=driver)
            info["Category"] = collection["Category"]
            info["Link"] = collection["Link"]
        
            collection_details.append(info)
            print(f"{index} Fetched details for {info['Title']}")
        except Exception as e:
            print(f"Error fetching details for {collection['Link']}: {e}")
    
    driver.close()
    driver.quit()
    
    return collection_details

In [10]:
def get_and_save_collection_details(collection_link_df, output_path):
    collection_details = get_collection_details(collection_link_df)
    
    collection_details_df = pd.DataFrame(collection_details)
    collection_details_df.to_csv(output_path, index=False)
    
    print(f"Saved collection details to {output_path}")
    
    return collection_details_df

In [18]:
total_chunk = 5

chunk_size = math.ceil(len(collection_links) / total_chunk)

chunks = [collection_links[i:i+chunk_size] for i in range(0, collection_links.shape[0], chunk_size)]

In [19]:
chunk_size

348

In [20]:
chunks[-1]

,Link,Category,Number Of Categories
1392,/lists/tfexka3n/song-of-ice-and-fire,"Literature,Just for Fun",2
1393,/lists/titdlevl/rise-and-fall-of-charles-lindb...,"Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade...",8
1394,/lists/tjkaza6a/5th-grade-math-terms,Grade 5,1
1395,/lists/tkllalrn/florida-collections-grade-7,Grade 12,1
1396,/lists/tl4r6xaq/camp-panda,Non-Fiction,1
...,...,...,...
1735,/lists/zx64qrvr/winesburg-ohio,"Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade...",8
1736,/lists/zxkf3gf4/no-summit-out-of-sight,Non-Fiction,1
1737,/lists/zxnoos5l/moonlit-vine,Literature,1
1738,/lists/zysetv6i/nyc-reads-365-grades-35,Non-Fiction,1


In [21]:
output_dir = "detail_chunk_output"

In [17]:
for index, p in enumerate(processes):
    p.terminate()
    print(f"Process {index} finished")

Process 0 finished
Process 1 finished


In [22]:
# drivers = [init_driver() for _ in range(len(chunks))]
# print(f"Initialized {len(drivers)} drivers")
processes = []

try:
    for index, chunk in enumerate(chunks):
        output_path = f"{output_dir}/chunk_{index}.csv"
        p = Process(target=get_and_save_collection_details, args=(chunk, output_path))
        processes.append(p)
        p.start()
        print(f"Started process for chunk {index}")
        sleep(2) # Wait for initializing the driver
        
    for index, p in enumerate(processes):
        p.join()
        print(f"Process {index} finished")
except Exception as e:
    for index, p in enumerate(processes):
        p.terminate()
        print(f"Process {index} finished")

Started process for chunk 0
Started process for chunk 1
Started process for chunk 2
0 Fetched details for Bunnicula
Started process for chunk 3
1 Fetched details for Ripped from the Headlines: February 2025
Started process for chunk 4
0 Fetched details for Shipwrecked!
2 Fetched details for Travel and Transportation
1 Fetched details for The GRE Verbal Reasoning Test: Challenge Words
0 Fetched details for The Ballad of Songbirds and Snakes
3 Fetched details for The Life and Crimes of Hoodie Rosen
2 Fetched details for Becoming
4 Fetched details for Saints of the Household
1 Fetched details for The Downstairs Girl
0 Fetched details for Once There Was
Error fetching details for /lists/tfexka3n/song-of-ice-and-fire: 'NoneType' object has no attribute 'select_one'
3 Fetched details for The Color of Water
2 Fetched details for Zara's Rules for Record-Breaking Fun
5 Fetched details for Absolutely Normal Chaos
Error fetching details for /lists/n23k4tyf/grades-1012: 'NoneType' object has no at

In [83]:
for index, p in enumerate(processes):
    p.terminate()
    print(f"Process {index} finished")

Process 0 finished
Process 1 finished


In [23]:
# Combine these chunked CSV files into a single CSV file
result_chunk_dfs = []
for index in range(len(chunks)):
    chunk_df = pd.read_csv(f"{output_dir}/chunk_{index}.csv")
    result_chunk_dfs.append(chunk_df)
    print(f"Loaded chunk {index} with {len(chunk_df)} rows")

Loaded chunk 0 with 330 rows
Loaded chunk 1 with 330 rows
Loaded chunk 2 with 333 rows
Loaded chunk 3 with 338 rows
Loaded chunk 4 with 332 rows


In [24]:
collection_infos = pd.concat(result_chunk_dfs, ignore_index=True)
collection_infos.head(2)

,Title,Subtitle,Description,Thumbnail Link,Book Purchase Link,Author,Category,Link
0,Bunnicula,NaN,"In this classic novel, Chester the cat is conv...",https://cdn.vocabulary.com/units/22f5x52u/cove...,https://www.amazon.com/dp/1416928170?tag=vocab...,Deborah Howe,"Grade 6,Grade 7,Grade 8,Literature,Just for Fun",/lists/22f5x52u/bunnicula
1,Ripped from the Headlines: February 2025,Our Weekly Roundups of Current Events Vocabulary,"Each week, we peruse the news to give you a wo...",https://cdn.vocabulary.com/units/23diw7dx/feat...,NaN,NaN,Events,/lists/23diw7dx/ripped-from-the-headlines-febr...


In [31]:
len(collection_infos)

1663

In [32]:
# Put a ID column
collection_infos["ID"] = collection_infos.index + 1
collection_infos.head(2)

,Title,Subtitle,Description,Thumbnail Link,Book Purchase Link,Author,Category,Link,ID
0,Bunnicula,NaN,"In this classic novel, Chester the cat is conv...",https://cdn.vocabulary.com/units/22f5x52u/cove...,https://www.amazon.com/dp/1416928170?tag=vocab...,Deborah Howe,"Grade 6,Grade 7,Grade 8,Literature,Just for Fun",/lists/22f5x52u/bunnicula,1
1,Ripped from the Headlines: February 2025,Our Weekly Roundups of Current Events Vocabulary,"Each week, we peruse the news to give you a wo...",https://cdn.vocabulary.com/units/23diw7dx/feat...,NaN,NaN,Events,/lists/23diw7dx/ripped-from-the-headlines-febr...,2


In [33]:
collection_infos.to_csv("collection_infos.csv", index=False)

# 3. Look insight the collection infos

In [34]:
collection_infos = pd.read_csv("collection_infos.csv")
print("Total records:", len(collection_infos))
collection_infos.head(2)

Total records: 1663


,Title,Subtitle,Description,Thumbnail Link,Book Purchase Link,Author,Category,Link,ID
0,Bunnicula,NaN,"In this classic novel, Chester the cat is conv...",https://cdn.vocabulary.com/units/22f5x52u/cove...,https://www.amazon.com/dp/1416928170?tag=vocab...,Deborah Howe,"Grade 6,Grade 7,Grade 8,Literature,Just for Fun",/lists/22f5x52u/bunnicula,1
1,Ripped from the Headlines: February 2025,Our Weekly Roundups of Current Events Vocabulary,"Each week, we peruse the news to give you a wo...",https://cdn.vocabulary.com/units/23diw7dx/feat...,NaN,NaN,Events,/lists/23diw7dx/ripped-from-the-headlines-febr...,2


In [35]:
collection_infos.isna().sum()

Title                    0
Subtitle              1324
Description              0
Thumbnail Link           0
Book Purchase Link     239
Author                 233
Category                 0
Link                     0
ID                       0
dtype: int64

In [ ]:
collection_details_df = pd.DataFrame(collection_details)